# Before Running the Script Please Make Sure you Select the Run Time as GPU

#**Clone the Github Repo**

In [1]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking


In [2]:
import os
#os.chdir(r"E:\school\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking")
print("Current Working Directory:", os.getcwd())  

os.chdir(os.path.join(os.getcwd(), "content", "YOLoV8-DeepSORT-Object-Tracking"))


Current Working Directory: E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'E:\\School\\YoloDeepSort\\YOLOv8-DeepSORT\\content\\YOLOv8-DeepSORT-Object-Tracking\\content\\YOLoV8-DeepSORT-Object-Tracking'

In [2]:
import torch
print("CUDA 是否可用:", torch.cuda.is_available())
print("CUDA 版本:", torch.version.cuda)
print("当前 GPU 数量:", torch.cuda.device_count())


CUDA 是否可用: True
CUDA 版本: 12.4
当前 GPU 数量: 1


##Setting the Current Directory

In [2]:
%cd E:\school\YoloDeepSort\YOLOv8-DeepSORT

E:\school\YoloDeepSort\YOLOv8-DeepSORT


In [3]:
%cd content\YOLOv8-DeepSORT-Object-Tracking\ultralytics\yolo\v8\detect

E:\school\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\ultralytics\yolo\v8\detect


In [ ]:
%cd content\YOLOv8-DeepSORT-Object-Tracking
!pip install -e '.[dev]'
#这些都在控制台执行比较好点，用于安装依赖

In [10]:
import ultralytics
print(ultralytics.__version__)

8.0.3


##Lets Run the Script 运行预测脚本

In [17]:
!python predict_customer.py source="test2.mp4" project="E:/School/YoloDeepSort/YOLOv8-DeepSORT/content" name="customerTest"
#write 


[2025-03-13 18:13:29,168][root.tracker][INFO] - Loading weights from deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7... Done!


E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\ultralytics\yolo\v8\detect\deep_sort_pytorch\deep_sort\deep\feature_extractor.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues rel

In [21]:
!python predict.py source="test1.mp4" 

save_dir is not a valid key. Similar keys: ['save_crop', 'save', 'save_hybrid']


In [21]:
!python predict.py source="test1.mp4" project="E:/School/YoloDeepSort/YOLOv8-DeepSORT/content" name="my_results"


[2025-03-10 15:57:12,927][root.tracker][INFO] - Loading weights from deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7... Done!


E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\ultralytics\yolo\v8\detect\deep_sort_pytorch\deep_sort\deep\feature_extractor.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues rel

##Display the Demo Video

In [22]:
import imageio_ffmpeg
import subprocess

# 获取 ffmpeg 可执行文件路径
ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()

save_path = r"E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\runs\detect\train7\test1.mp4"
compressed_path = r"E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\result_compressed1.mp4"

ffmpeg_command = f'"{ffmpeg_path}" -i "{save_path}" -vcodec libx264 "{compressed_path}"'
print("正在执行:", ffmpeg_command)

# 运行 ffmpeg
result = subprocess.run(ffmpeg_command, shell=True, capture_output=True, text=True)

# 打印输出
print("标准输出:", result.stdout)
print("错误信息:", result.stderr)


正在执行: "F:\anoconda\envs\YOLOv8-DeepSORT\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe" -i "E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\runs\detect\train7\test1.mp4" -vcodec libx264 "E:\School\YoloDeepSort\YOLOv8-DeepSORT\content\result_compressed1.mp4"
标准输出: 
错误信息: ffmpeg version 7.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libh

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = r"E:\学校\YoloDeepSort\YOLOv8-DeepSORT\content\YOLOv8-DeepSORT-Object-Tracking\runs\detect\train4\test1.mp4"

# Compressed video path
compressed_path = r"E:\学校\YoloDeepSort\YOLOv8-DeepSORT\content\result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#查看追踪的视频
import cv2

video_path = r"E:\学校\YoloDeepSort\YOLOv8-DeepSORT\content\result_compressed.mp4"

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("⚠️ 无法打开视频文件！")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break  # 读取完毕
    cv2.imshow("Video Playback", frame)

    # 按 `q` 退出播放
    if cv2.waitKey(25) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
